This function computes the tensorlines of the eigenvector-field $ \mathbf{\xi}_i(\mathbf{x}) $. Due to the unavoidable discontinuities of the eigenvector-field, the eigenvectors have to be re-oriented on the fly before each integration step.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| eig | array (Ny, Nx) | eigenvalue-field $ \mathbf{\lambda_i}(\mathbf{x}), \quad i = \lbrace 1, 2 \rbrace $  |
| eig_opposite | array (Ny, Nx) | eigenvalue-field $ \mathbf{\lambda_{-i}}(\mathbf{x}), \quad i = \lbrace 1, 2 \rbrace $  |
| vector_field | array (Ny, Nx, 2) | eigenvector-field $ \mathbf{\xi_i}(\mathbf{x}), \quad i = \lbrace 1, 2 \rbrace $|
| min_distance | float | minimum distance between local maxima |
| max_length | float | maxim length of tensorline |
| step_size | float | integration step size |
| n_tensorlines | int | number of most relevant tensorlines; if n_tensorlines == -1 then all tensorlines are computed|
| hyperbolicity | float | stop integration of tensorlines if rate of attraction/repulsion < hyperbolicity|
| type | string | "shrink" --> compute shrinklines; "stretch" --> compute stretchlines|
| n_iterations | float | stop integration of tensorlines if number of iterations > n_iterations|

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/Src/utils")

In [2]:
# Import numpy
import numpy as np

# import math tools
from math import sqrt

# import progress bar
from tqdm.notebook import tqdm

# import function to compute local maxima
from ipynb.fs.defs.loc_max import _loc_max

# import function to orient vectorfield
from ipynb.fs.defs.orient_vectorfield import _orient_vectorfield

# import integrator for tensorlines
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines_incompressible, _RK4_tensorlines_compressible

# Import eigenvalues/eigenvectors calculator
from ipynb.fs.defs.eigen import eigen

# Import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [3]:
def _tensorlines_incompressible(X, Y, eig, vector_field, min_distance, max_length, step_size, n_tensorlines = -1, hyperbolicity = 0, n_iterations = 10**4, verbose = False):
    '''
    Wrapper for RK4_tensorlines_incompressible(). Integrates the tensorlines given an eigenvector/eigenvalue field. The integration stops
    whenever a threshold value is reached (max_length/hyperbolicity/counter_threshold).
    
    Parameters:
        X:                 array (Ny, Nx), X-meshgrid (of complete data domain).
        Y:                 array (Ny, Nx), Y-meshgrid (of complete data domain).
        eig:               array (Ny, Nx), eigenvalue field.
        vector_field:      array (Ny, Nx, 2), eigenvector field.
        min_distance:      float, minimum distance between local extrema in the eigenvalue field.
        max_length:        float, maximum length of tensorlines.
        step_size:         float, step size used for integration. This value is kept constant.
        n_tensorlines:     int, extract only the most relevant tensorlines. If n_tensorlines = -1, then algorithm returns all possible tensorlines.
        hyperbolicity:     float, threshold on hyperbolicity value. 
                           If point on tensorline has hyperbolicity lower than this threshold, than integration is stop.
        n_iterations:      int, threshold on number of iterations.
        verbose:           bool, if True, function reports progress at every 100th iteration
    
    Returns:
        tensorlines:       list, each element in the list contains an array specifying the x/y-coordinates of the tensorlines.
    '''
    
    # Find local extrema of eigfield
    peak_x, peak_y, peak_field = _loc_max(min_distance, X, Y, eig)[2:]
    
    # if you do not place a restriction on the number of tensorlines, 
    # then as many tensorlines as possible are plotted
    if n_tensorlines == -1:
        n_tensorlines = len(peak_field)
    
    # defined domain
    defined_domain = np.isfinite(eig).astype(int)
    
    # set nan values of eig to zero for gridded interpolation
    eig = np.nan_to_num(eig, 0)
        
    # Interpolate Eigenvalue field
    interp_eig = RBS(Y[:,0], X[0,:], eig, kx = 1, ky = 1)
    
    # Define list of tensorlines (back/forward)
    tensorlines = [[], []]
    
    # Iterate over all local maxima
    for i in range(len(peak_x[:n_tensorlines])):
        
        tensorlines_forw = [[], []]
        tensorlines_back = [[], []]
        
        # Local maxima point
        x = np.array([peak_x[i], peak_y[i]])
        
        if peak_field[i] > hyperbolicity:
            # Boolean forward Iteration
            bool_forward, bool_backward = True, True
        
        else:
            bool_forward, bool_backward = False, False
        
        # Start integration only if local maxima is not close than 'min_distance' to shrinkline
        if bool_forward and bool_backward:
            
            # Starting point of integration
            x_forward = x
            x_backward = x
            
            # Append starting point to list containing positions of forward shrinklines
            for ii in range(2):
                tensorlines_forw[ii].append(x[ii])
                
            # Check orientation of vector-field and rieorient if needed.
            vx, vy = _orient_vectorfield(X, Y, x, vector_field)
            
            # Initial vector orientation
            x_prime_forward = np.array([vx, vy])
            x_prime_backward = -np.array([vx, vy])
            
            # Initial distance
            dist = 0
            
            # set counter for number of iterations
            counter = 0
            
            # Start Integration with dummy variable 's_array'
            if verbose:
                pbar = tqdm(desc='while loop', total=n_iterations)
            while (bool_forward or bool_backward) and counter < n_iterations:
                
                #if verbose and counter%100==0:
                #    print("Percentage completed: ", np.around(counter/n_iterations, int(np.log(n_iterations))+1)*100)
                
                # Integrate only if 'x_prime_forward' is defined and 'bool_forward == True'
                if bool_forward and x_prime_forward is not None:
                    
                    # RK4 integration for tensorline
                    x_forward, x_prime_forward = _RK4_tensorlines_incompressible(X, Y, defined_domain, x_forward, x_prime_forward, step_size, vector_field, interp_eig) 
                        
                    if x_forward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_forw[ii].append(x_forward[ii])
                                
                        else:
                            
                            bool_forward = False
                        
                    else:
                        
                        bool_forward = False
                
                # Integrate only if 'x_prime_backward' is defined and 'bool_backward == True'
                if bool_backward and x_prime_backward is not None:
                    
                    # RK4 integration for tensorline
                    x_backward, x_prime_backward = _RK4_tensorlines_incompressible(X, Y, defined_domain, x_backward, x_prime_backward, step_size, vector_field, interp_eig) 
                    
                    if x_backward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_back[ii].append(x_backward[ii])
                    
                        else:
                            
                            bool_backward = False
                    
                    else:
                        
                        bool_backward = False
                        
                counter += 1
                
                if verbose:
                    pbar.update(counter)
                        
            # Append backward and forward shrinkline
            for ii in range(2):
                tensorlines[ii].append(np.append(np.flip(tensorlines_back[ii]), tensorlines_forw[ii]))
            
            if verbose:
                pbar.close()
    
    return tensorlines

In [ ]:
def _tensorlines_compressible(X, Y, eig, eig_opposite, vector_field, min_distance, max_length, step_size, n_tensorlines = -1, hyperbolicity = 0, type = "shrink", n_iterations = 10**4, verbose = False):
    '''
    Wrapper for RK4_tensorlines_compressible(). Integrates the tensorlines for a compressible velocity field given an eigenvector/eigenvalue field. The integration stops
    whenever a threshold value is reached (max_length/hyperbolicity/counter_threshold).
    
    Parameters:
        X:             array (Ny, Nx), X-meshgrid (of complete data domain).
        Y:             array (Ny, Nx), Y-meshgrid (of complete data domain).
        eig:           array (Ny, Nx), eigenvalue field.
        eig_opposite:  array (Ny, Nx), eigenvalue field opposite to eig.
        vector_field:  array (Ny, Nx, 2), eigenvector field.
        min_distance:  float, minimum distance between local extrema in the eigenvalue field.
        max_length:    float, maximum length of tensorlines.
        step_size:     float, step size used for integration. This value is kept constant.
        n_tensorlines: int, extract only the most relevant tensorlines. If n_tensorlines = -1, then algorithm returns all possible tensorlines.
        hyperbolicity: float, threshold on hyperbolicity value. 
                       If point on tensorline has hyperbolicity lower than this threshold, than integration is stop.
        type:          bool, type of tensorlines. "stretch" --> stretchline or "shrink" --> shrinkline.
        n_iterations:  int, threshold on number of iterations.
        verbose:       bool, if True, function reports progress at every 100th iteration
    
    Returns:
        tensorlines:   list, each element in the list contains an array specifying the x/y-coordinates of the tensorlines.
    '''
    
    if type == "stretch":
        
        sign = -1
        
        # Find local extrema of eigfield
        peak_x, peak_y, peak_field = _loc_max(min_distance, X, Y, 1-eig, 1-hyperbolicity)[2:]
    
        peak_field = 1-peak_field
        
    elif type == "shrink":
        
        sign = 1
        
        # Find local extrema of eigfield
        peak_x, peak_y, peak_field = _loc_max(min_distance, X, Y, eig, hyperbolicity)[2:]
    
    if n_tensorlines == -1:
        n_tensorlines = len(peak_field)

    eig = sign*eig
    eig_opposite = sign*eig_opposite
    
    # defined domain
    defined_domain = np.isfinite(eig).astype(int)
    
    # set nan values of eig to zero for gridded interpolation
    eig = np.nan_to_num(eig, 0)
    
    # set nan values of eig to zero for gridded interpolation
    eig_opposite = np.nan_to_num(eig_opposite, 0)
        
    # Interpolate Eigenvalue field
    interp_eig = RBS(Y[:,0], X[0,:], eig, kx = 1, ky = 1)
    
    # Interpolate Eigenvalue field
    interp_eig_opposite = RBS(Y[:,0], X[0,:], eig_opposite, kx = 1, ky = 1)
    
    # Define list of tensorlines (back/forward)
    tensorlines = [[], []]
    
    peak_return_x, peak_return_y, peak_return_value = [], [], []
    
    # Iterate over all local maxima
    for i in tqdm(range(len(peak_x[:n_tensorlines]))):
        
        tensorlines_forw = [[], []]
        tensorlines_back = [[], []]
        
        # Local maxima point
        x = np.array([peak_x[i], peak_y[i]])
        
        if sign*peak_field[i] >= sign*hyperbolicity:
            # Boolean forward Iteration
            bool_forward, bool_backward = True, True
        
        else:
            bool_forward, bool_backward = False, False
        
        # Start integration only if local maxima is not closer than 'min_distance' to shrinkline
        if bool_forward and bool_backward:
            
            peak_return_x.append(peak_x[i])
            peak_return_y.append(peak_y[i])
            peak_return_value.append(peak_field[i])
            
            # Starting point of integration
            x_forward = x
            x_backward = x
            
            # Append starting point to list containing positions of forward shrinklines
            for ii in range(2):
                tensorlines_forw[ii].append(x[ii])
                
            # Check orientation of vector-field and rieorient if needed.
            vx, vy = _orient_vectorfield(X, Y, x, vector_field)
            
            # Initial vector orientation
            x_prime_forward = np.array([vx, vy])
            x_prime_backward = -np.array([vx, vy])
            
            # Initial distance
            dist = 0
            
            # set counter to limit unusually many iterations
            counter = 0
            
            # Start Integration with dummy variable 's_array'
            if verbose:
                pbar = tqdm(desc='while loop', total=n_iterations)
                
            while (bool_forward or bool_backward) and counter < n_iterations:
                
                # Integrate only if 'x_prime_forward' is defined and 'bool_forward == True'
                if bool_forward and x_prime_forward is not None:
                    
                    # RK4 integration for tensorline
                    x_forward, x_prime_forward = _RK4_tensorlines_compressible(X, Y, defined_domain, x_forward, x_prime_forward, step_size, vector_field, interp_eig, interp_eig_opposite) 
                        
                    if x_forward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_forw[ii].append(x_forward[ii])
                                
                        else:
                            
                            bool_forward = False
                        
                    else:
                        
                        bool_forward = False
                
                # Integrate only if 'x_prime_backward' is defined and 'bool_backward == True'
                if bool_backward and x_prime_backward is not None:
                    
                    # RK4 integration for tensorline
                    x_backward, x_prime_backward = _RK4_tensorlines_compressible(X, Y, defined_domain, x_backward, x_prime_backward, step_size, vector_field, interp_eig, interp_eig_opposite) 
                    
                    if x_backward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_back[ii].append(x_backward[ii])
                    
                        else:
                            
                            bool_backward = False
                    
                    else:
                        
                        bool_backward = False
                        
                counter += 1
                
                if verbose:
                    pbar.update(counter)
                        
            # Append backward and forward shrinkline
            for ii in range(2):
                tensorlines[ii].append(np.append(np.flip(tensorlines_back[ii]), tensorlines_forw[ii]))
        
        if verbose:
            pbar.close()
    
    return tensorlines, peak_return_x, peak_return_y, peak_return_value